# Teste de modelos

Este notebook contém os modelos a serem avaliados utilizando os atributos considerados pelo relatório da Organização Mundial da Saúde.

In [1]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [2]:
# Carregando os dados
data = pd.read_excel('../../Dados/Banco labels.xlsx')

In [3]:
# Visualização da tabela
data.head(5)

,TB_desfecho_final_Dri,Tbafranio,Obs_desfechoTB_Dri_Dani,Data_inclusão,Nome,Desfecho_tuberculose_lista_Mauro,Desfecho_TB_162_Revisão_ADRIANA,Observaça0_desfecho_TB,Resultado_BAAR_1,Resultado_BAAR_2,...,Perdeu_aula_hoje_CA,Deixou_de_ganhar_dinheiro_hoje_CA,Quanto_deixou_ganhar_CA,Caso_acompanhado_quem_custeou_tudo_CA,Valor_CA,Avaliador_custo_CA,Data_avaliação_custo_CA,Resultado_cultura_final < 3 & Realizou_RX = 1 (FILTER),ANO de inclusao,Novocriteriotbafranio
0,TB,SIM,NaN,2008-04-03,NaN,9,TB,TB PROBABILIDADE,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2008,1
1,TB,SIM,NaN,2006-10-03,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2006,1
2,TB,SIM,NaN,2007-06-21,NaN,TB,TB,TB CONFIRMADA,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2007,1
3,TB,SIM,NaN,2009-09-30,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2009,1
4,TB,SIM,NaN,2009-06-19,NaN,TB,NaN,NaN,Negativo,Negativo,...,Não,Não,"""Não se aplica""",não se aplica,não se aplica,Marcia dos Santos,2009-06-19,Not Selected,2009,1


## Sintomas considerados pelo WHO

De acordo com o [WHO](http://www.who.int/news-room/fact-sheets/detail/tuberculosis):

"Common symptoms of active lung TB are cough with sputum and blood at times, chest pains, weakness, weight loss, fever and night sweats. Many countries still rely on a long-used method called sputum smear microscopy to diagnose TB. Trained laboratory technicians look at sputum samples under a microscope to see if TB bacteria are present. Microscopy detects only half the number of TB cases and cannot detect drug-resistance."

In [4]:
# Atributos considerados pelo WHO
features_selecionadas = ['TB_desfecho_final_Dri','Expectoração','Tosse','Hemoptise','Dor_torácica',
                         'Perda_peso_10percent','Sudorese_noturna','Febre']

# Dados filtrados
dados = data[(data.TB_desfecho_final_Dri =='TB') | (data.TB_desfecho_final_Dri =='Não TB')][features_selecionadas].copy()

In [5]:
# Visualização dos dados
dados.head(5)

,TB_desfecho_final_Dri,Expectoração,Tosse,Hemoptise,Dor_torácica,Perda_peso_10percent,Sudorese_noturna,Febre
0,TB,Não,Sim,Não,Não,Não,Não,Não
1,TB,Sim,Sim,Não,Não,Não,Não,Não
2,TB,Sim,Sim,Não,Sim,Sim,Não,Sim
3,TB,Não,Sim,Não,Sim,Não,Não,Não
4,TB,Sim,Sim,Não,Sim,Não,Sim,Não


In [12]:
dados.shape

(2727, 8)

In [9]:
# Verificando os valores presentes para cada feature

for feature in list(dados.columns):
    print('Valores presentes da feature ' + feature)
    print(dados[feature].value_counts())
    print()

Valores presentes da feature desfecho
TB-    1856
TB+     871
Name: desfecho, dtype: int64

Valores presentes da feature Expectoração
Sim         2155
Não          567
ignorado       5
Name: Expectoração, dtype: int64

Valores presentes da feature Tosse
Sim         2447
Não          279
ignorado       1
Name: Tosse, dtype: int64

Valores presentes da feature Hemoptise
Não         2465
Sim          242
ignorado      20
Name: Hemoptise, dtype: int64

Valores presentes da feature Dor_torácica
Sim         1918
Não          806
ignorado       3
Name: Dor_torácica, dtype: int64

Valores presentes da feature Perda_peso_10percent
Não         1862
Sim          721
ignorado     144
Name: Perda_peso_10percent, dtype: int64

Valores presentes da feature Sudorese_noturna
Não         1720
Sim          998
ignorado       9
Name: Sudorese_noturna, dtype: int64

Valores presentes da feature Febre
Não         1466
Sim         1251
ignorado      10
Name: Febre, dtype: int64



## 1. Pré-processamento dos dados

In [10]:
# Alteração do label da variável alvo
dados.rename(index=str,columns={'TB_desfecho_final_Dri':'desfecho'},inplace=True)

# Alteração dos labels presentes do desfecho
dados.replace(['TB','Não TB','IGN',np.nan,8,9],['TB+','TB-','ignorado','ignorado','ignorado','ignorado'],inplace=True)

In [11]:
# Alteração dos dados 
dados.replace(['IGN',8,9],['ignorado','ignorado','ignorado'],inplace=True)

# Quantidade de dados ignorados
print('Quantidade de dados faltantes para cada feature:\n')

for feature in list(dados.columns)[1:]:
    print('Feature {}: {} dados ignorados'.format(feature,dados[feature].value_counts().loc['ignorado']))

Quantidade de dados faltantes para cada feature:

Feature Expectoração: 5 dados ignorados
Feature Tosse: 1 dados ignorados
Feature Hemoptise: 20 dados ignorados
Feature Dor_torácica: 3 dados ignorados
Feature Perda_peso_10percent: 144 dados ignorados
Feature Sudorese_noturna: 9 dados ignorados
Feature Febre: 10 dados ignorados


In [13]:
dados.head(5)

,desfecho,Expectoração,Tosse,Hemoptise,Dor_torácica,Perda_peso_10percent,Sudorese_noturna,Febre
0,TB+,Não,Sim,Não,Não,Não,Não,Não
1,TB+,Sim,Sim,Não,Não,Não,Não,Não
2,TB+,Sim,Sim,Não,Sim,Sim,Não,Sim
3,TB+,Não,Sim,Não,Sim,Não,Não,Não
4,TB+,Sim,Sim,Não,Sim,Não,Sim,Não


In [8]:
# Alterando os valores para nan  
dados.replace(['ignorado'],[np.nan],inplace=True)

# Quantas linhas possuem valores ignorados
# dados_modelo = dados.dropna(how='any')

In [14]:
# Realizando o pré-processamento descrito (-1,0,1)
dados.replace(['Não','Sim','ignorado'],[-1,0,1],inplace=True)

In [9]:
# Cálculo de Odds e Odds Ratio, seguindo a tabela usando A,B,C,D
odds_ratio = dict()

for feature in list(dados_modelo.columns)[1:]:
    
    # Cálculo de Odds positivo : Razão dos casos para o caso do sintoma presente
    a = len(dados_modelo[(dados_modelo.desfecho == 'TB+') & (dados_modelo[feature] == 'Sim')])
    c = len(dados_modelo[(dados_modelo.desfecho == 'TB-') & (dados_modelo[feature] == 'Sim')])
    odds_positivo = a/c
    
    # Cálculo de Odds negativo : Razão dos casos para o caso do sintoma ausente
    b = len(dados_modelo[(dados_modelo.desfecho == 'TB+') & (dados_modelo[feature] == 'Não')])
    d = len(dados_modelo[(dados_modelo.desfecho == 'TB-') & (dados_modelo[feature] == 'Não')])
    odds_negativo = b/d
    
    odds_ratio[feature] = (odds_positivo) / (odds_negativo)

In [10]:
# Criando DataFrame para Odds Ratio

odds_table = pd.DataFrame.from_dict(odds_ratio,orient='index')
odds_table.rename(index=str,columns={0 : 'Odds_Ratio'},inplace=True)
odds_table.T

,Sudorese_noturna,Perda_peso_10percent,Expectoração,Febre,Tosse,Dor_torácica,Hemoptise
Odds_Ratio,1.812336,3.616097,0.895358,2.274124,1.365097,1.025702,1.497173


In [16]:
# Desbalanceamento existente entre as classes TB+ e TB-

tbp = len(dados[dados.desfecho == 'TB+'])
tbn = len(dados[dados.desfecho == 'TB-'])

print('Quantidade de exemplos TB+: {}  {:.2f}%\nQuantidade de exemplos TB-: {} {:.2f}%'.format(tbp, 100 * tbp/dados.shape[0], 
                                                                                          tbn, 100 * tbn/dados.shape[0]))


Quantidade de exemplos TB+: 871  31.94%
Quantidade de exemplos TB-: 1856 68.06%


## 2. Teste de Modelos

#### 2.1 Preparação dos dados para aplicação dos modelos

In [17]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression as LG

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing   import LabelEncoder

# Métricas para avaliação dos modelos
from sklearn.metrics         import roc_auc_score, f1_score, confusion_matrix

In [22]:
# Renomeando os labels existentes do dataframe
dados_modelo = dados_modelo.apply(LabelEncoder().fit_transform)

In [19]:
# Separando o target dos dados
target        = dados.desfecho
dados_modelo  = dados.drop(['desfecho'],axis=1)

In [22]:
target.replace(['TB+','TB-'],[1,0],inplace=True)

In [23]:
dados_modelo.head(5)

,Expectoração,Tosse,Hemoptise,Dor_torácica,Perda_peso_10percent,Sudorese_noturna,Febre
0,-1,0,-1,-1,-1,-1,-1
1,0,0,-1,-1,-1,-1,-1
2,0,0,-1,0,0,-1,0
3,-1,0,-1,0,-1,-1,-1
4,0,0,-1,0,-1,0,-1


In [25]:
cv = cross_val_score(LG(C=10000),dados_modelo,target,scoring='roc_auc',cv=10,n_jobs=-1)

In [26]:
cv

array([ 0.56708211,  0.69839946,  0.73893833,  0.69435175,  0.64454332,
        0.69892473,  0.65051258,  0.71997515,  0.66486486,  0.55262504])

In [27]:
print(np.mean(cv), ' +/- ',np.std(cv))

0.663021732674  +/-  0.0586705785764
